# wb-point-v1 UDP TX Rate Test\n\nNotebook sender for Wand-Brain UDP input (`wb-point-v1`).\nRuns a rate sweep at 25/50/100 packets per second.

In [ ]:
import math\nimport socket\nimport struct\nimport time\nfrom dataclasses import dataclass

In [ ]:
# ===== Config =====\nHOST = "127.0.0.1"        # change to EC2 IP, e.g. "16.16.202.231"\nPORT = 41000\nDEVICE_NUMBER = 1\nWAND_ID = 1\nSTROKE_START = 2000\nPACKET_START = 0\nRATES = [25, 50, 100]\nDURATION_S = 3.0\nPAUSE_S = 0.5\nPATTERN = "line"  # "line" or "circle"

In [ ]:
# ===== Protocol =====\nWB_STRUCT = struct.Struct("<HBBHHIIhhI")\nMAGIC = 0x5742\nVERSION = 1\nPEN_DOWN = 0x01\nSTROKE_START_FLAG = 0x02\nSTROKE_END = 0x04\nQ15_MAX = 32767\n\ndef now_ms_u32():\n    return int(time.time() * 1000) & 0xFFFFFFFF\n\ndef to_q15(v):\n    return max(0, min(Q15_MAX, int(round(v * Q15_MAX))))\n\ndef point_at(i, n, pattern="line"):\n    if n <= 1:\n        return 0.5, 0.5\n    t = i / (n - 1)\n    if pattern == "circle":\n        a = 2.0 * math.pi * t\n        return 0.5 + 0.45 * math.cos(a), 0.5 + 0.45 * math.sin(a)\n    return t, t

In [ ]:
@dataclass\nclass TxStats:\n    rate_hz: float\n    target_points: int\n    sent_points: int\n    achieved_pps: float\n    mean_dt_ms: float

In [ ]:
def send_stroke(sock, host, port, *, device_number, wand_id, stroke_id, packet_number_start, rate_hz, duration_s, pattern):\n    n_points = max(2, int(round(rate_hz * duration_s)))\n    packet_number = packet_number_start\n    send_times = []\n    start = time.perf_counter()\n    last_x_q, last_y_q = to_q15(0.5), to_q15(0.5)\n\n    for i in range(n_points):\n        target_t = start + (i / rate_hz)\n        while True:\n            now = time.perf_counter()\n            rem = target_t - now\n            if rem <= 0:\n                break\n            time.sleep(min(rem, 0.001))\n\n        x, y = point_at(i, n_points, pattern=pattern)\n        x_q, y_q = to_q15(x), to_q15(y)\n        flags = PEN_DOWN | (STROKE_START_FLAG if i == 0 else 0)\n\n        payload = WB_STRUCT.pack(\n            MAGIC, VERSION, flags,\n            device_number, wand_id,\n            packet_number, stroke_id,\n            x_q, y_q, now_ms_u32()\n        )\n        sock.sendto(payload, (host, port))\n        send_times.append(time.perf_counter())\n        packet_number += 1\n        last_x_q, last_y_q = x_q, y_q\n\n    # final packet: STROKE_END with PEN_DOWN=0\n    end_payload = WB_STRUCT.pack(\n        MAGIC, VERSION, STROKE_END,\n        device_number, wand_id,\n        packet_number, stroke_id,\n        last_x_q, last_y_q, now_ms_u32()\n    )\n    sock.sendto(end_payload, (host, port))\n    packet_number += 1\n\n    if len(send_times) > 1:\n        elapsed = max(send_times[-1] - send_times[0], 1e-9)\n        achieved_pps = (len(send_times) - 1) / elapsed\n        dts_ms = [(send_times[i] - send_times[i - 1]) * 1000.0 for i in range(1, len(send_times))]\n        mean_dt_ms = sum(dts_ms) / len(dts_ms)\n    else:\n        achieved_pps = 0.0\n        mean_dt_ms = 0.0\n\n    return packet_number, TxStats(rate_hz, n_points, n_points, achieved_pps, mean_dt_ms)

In [ ]:
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)\npacket_number = PACKET_START\nstroke_id = STROKE_START\n\nprint(f"Target: {HOST}:{PORT} device={DEVICE_NUMBER} wand={WAND_ID}")\nfor i, rate in enumerate(RATES, start=1):\n    packet_number, st = send_stroke(\n        sock, HOST, PORT,\n        device_number=DEVICE_NUMBER,\n        wand_id=WAND_ID,\n        stroke_id=stroke_id,\n        packet_number_start=packet_number,\n        rate_hz=float(rate),\n        duration_s=DURATION_S,\n        pattern=PATTERN,\n    )\n    print(f"[{i}/{len(RATES)}] stroke_id={stroke_id} rate={st.rate_hz:.1f} sent={st.sent_points} achieved~{st.achieved_pps:.2f} pps mean_dt={st.mean_dt_ms:.2f} ms")\n    stroke_id += 1\n    if i < len(RATES):\n        time.sleep(PAUSE_S)\n\nprint("Done.")